amount_tsh <span style="color: orange;">(Numerical)</span>- Total static head (amount water available to waterpoint)

date_recorded <span style="color: blue;">(Date)</span>- The date the row was entered

funder <span style="color: green;">(Categorical)</span>- Who funded the well

gps_height <span style="color: orange;">(Numerical)</span>- Altitude of the well

installer <span style="color: green;">(Categorical)</span>- Organization that installed the well

longitude <span style="color: orange;">(Numerical)</span>- GPS coordinate

latitude <span style="color: orange;">(Numerical)</span>- GPS coordinate

wpt_name <span style="color: green;">(Categorical)</span>- Name of the waterpoint if there is one

num_private <span style="color: orange;">(Numerical)</span>

basin <span style="color: green;">(Categorical)</span>- Geographic water basin

subvillage <span style="color: green;">(Categorical)</span>- Geographic location

region <span style="color: green;">(Categorical)</span>- Geographic location

region_code <span style="color: green;">(Categorical)</span>- Geographic location (coded)

district_code <span style="color: green;">(Categorical)</span>- Geographic location (coded)

lga <span style="color: green;">(Categorical)</span>- Geographic location

ward <span style="color: green;">(Categorical)</span>- Geographic location

population <span style="color: orange;">(Numerical)</span>- Population around the well

public_meeting <span style="color: purple;">(Binary)</span>- True/False

recorded_by <span style="color: green;">(Categorical)</span>- Group entering this row of data

scheme_management <span style="color: green;">(Categorical)</span>- Who operates the waterpoint

scheme_name <span style="color: green;">(Categorical)</span>- Who operates the waterpoint

permit <span style="color: purple;">(Binary)</span>- If the waterpoint is permitted

construction_year <span style="color: blue;">(Date)</span>- Year the waterpoint was constructed

extraction_type <span style="color: green;">(Categorical)</span>- The kind of extraction the waterpoint uses

extraction_type_group <span style="color: green;">(Categorical)</span>- The kind of extraction the waterpoint uses

extraction_type_class <span style="color: green;">(Categorical)</span>- The kind of extraction the waterpoint uses

management <span style="color: green;">(Categorical)</span>- How the waterpoint is managed

management_group <span style="color: green;">(Categorical)</span>- How the waterpoint is managed

payment <span style="color: green;">(Categorical)</span>- What the water costs

payment_type <span style="color: green;">(Categorical)</span>- What the water costs

water_quality <span style="color: green;">(Categorical)</span>- The quality of the water

quality_group <span style="color: green;">(Categorical)</span>- The quality of the water

quantity <span style="color: green;">(Categorical)</span>- The quantity of water

quantity_group <span style="color: green;">(Categorical)</span>- The quantity of water

source <span style="color: green;">(Categorical)</span>- The source of the water

source_type <span style="color: green;">(Categorical)</span>- The source of the water

source_class <span style="color: green;">(Categorical)</span>- The source of the water

waterpoint_type <span style="color: green;">(Categorical)</span>- The kind of waterpoint

waterpoint_type_group <span style="color: green;">(Categorical)</span>- The kind of waterpoint

<span style="color: red;">Objective variable:</span> status_group <span style="color: green;">(Categorical)</span>

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
from scipy import cluster
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
root_repo = 'https://raw.githubusercontent.com/JaviARo/Datasets/refs/heads/main/WaterTable/'

train_labels_repo, train_values_repo, test_values_repo = root_repo + 'train_labels.csv', root_repo + 'train_values.csv', root_repo + 'test_values.csv'

train_labels = pd.read_csv(train_labels_repo, sep=',')
train_values = pd.read_csv(train_values_repo, sep=',')
test_values = pd.read_csv(test_values_repo, sep=',')

In [3]:
train_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [4]:
train_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
test_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [6]:
for column in train_values.columns:
  num_valores_unicos = train_values[column].nunique()
  print(f'Unique values in column {column}: {num_valores_unicos}')

Unique values in column id: 59400
Unique values in column amount_tsh: 98
Unique values in column date_recorded: 356
Unique values in column funder: 1896
Unique values in column gps_height: 2428
Unique values in column installer: 2145
Unique values in column longitude: 57516
Unique values in column latitude: 57517
Unique values in column wpt_name: 37399
Unique values in column num_private: 65
Unique values in column basin: 9
Unique values in column subvillage: 19287
Unique values in column region: 21
Unique values in column region_code: 27
Unique values in column district_code: 20
Unique values in column lga: 125
Unique values in column ward: 2092
Unique values in column population: 1049
Unique values in column public_meeting: 2
Unique values in column recorded_by: 1
Unique values in column scheme_management: 11
Unique values in column scheme_name: 2695
Unique values in column permit: 2
Unique values in column construction_year: 55
Unique values in column extraction_type: 18
Unique valu

In [6]:
le = LabelEncoder()

pd.options.mode.copy_on_write = True

categorical_columns = [
  'funder',
	'installer',
	'wpt_name',
	'basin',
	'subvillage',
	'region',
	'region_code',
	'district_code',
	'lga',
	'ward',
	'recorded_by',
	'scheme_management',
	'scheme_name',
	'extraction_type',
	'extraction_type_group',
	'extraction_type_class',
	'management',
	'management_group',
	'payment',
	'payment_type',
	'water_quality',
	'quality_group',
	'quantity',
	'quantity_group',
	'source',
	'source_type',
	'source_class',
	'waterpoint_type',
	'waterpoint_type_group'
]

for column in categorical_columns: 
	train_values[column] = le.fit_transform(train_values[column])

In [23]:
binary_columns = ['public_meeting', 'permit']

for column in binary_columns:
  train_values[column].replace(['', 'False', 'True'],[0,0,1])

In [8]:
train_values['date_recorded'] = train_values['date_recorded'].str.replace('-', '')

In [9]:
train_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,20110314,1368,1390,1518,34.938093,-9.856322,37398,0,...,0,6,2,1,1,8,6,0,1,1
1,8776,0.0,20130306,469,1399,545,34.698766,-2.147466,37194,0,...,2,6,2,2,2,5,3,1,1,1
2,34310,25.0,20130225,825,686,2048,37.460664,-3.821329,14572,0,...,5,6,2,1,1,0,1,1,2,1
3,67743,0.0,20130128,1740,263,1852,38.486161,-11.155298,37284,0,...,2,6,2,0,0,3,0,0,2,1
4,19728,0.0,20110713,20,0,119,31.130847,-1.825359,35528,0,...,2,6,2,3,3,5,3,1,1,1


In [10]:
min_max_scaler = preprocessing.MinMaxScaler()
train_values_scaled = min_max_scaler.fit_transform(train_values)
train_values_scaled

array([[9.37034493e-01, 1.71428571e-02, 8.10425723e-01, ...,
        0.00000000e+00, 1.66666667e-01, 2.00000000e-01],
       [1.18200062e-01, 0.00000000e+00, 9.91859442e-01, ...,
        5.00000000e-01, 1.66666667e-01, 2.00000000e-01],
       [4.62106213e-01, 7.14285714e-05, 9.91124341e-01, ...,
        5.00000000e-01, 3.33333333e-01, 2.00000000e-01],
       ...,
       [4.99104341e-01, 0.00000000e+00, 8.11306029e-01, ...,
        0.00000000e+00, 6.66666667e-01, 6.00000000e-01],
       [4.21323420e-01, 0.00000000e+00, 8.10371271e-01, ...,
        0.00000000e+00, 6.66666667e-01, 6.00000000e-01],
       [3.54869557e-01, 0.00000000e+00, 8.10507401e-01, ...,
        0.00000000e+00, 6.66666667e-01, 6.00000000e-01]])

In [12]:
X_train, X_val, y_train, y_val = train_test_split(train_values_scaled, train_labels['status_group'], test_size=0.20, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [14]:
X_train.shape

(35640, 40)

In [15]:
y_train.shape

(35640,)

In [17]:
def PlotMatrizCorrelacion(corr_mat_data):
	sns.set_theme()
	data_mask = np.triu(np.ones_like(corr_mat_data, dtype = bool)) # Recorta la gráfica

	cmap = sns.diverging_palette(100, 7, s = 75, l = 40, n = 5, center = 'light', as_cmap = True)

	plt.figure(figsize = (23, 15))
	# sns.heatmap(X.corr(), square=True, annot=True)
	sns.heatmap(corr_mat_data, annot = True, cmap = cmap, fmt = '.2f', center = 0,
							annot_kws = {'size': 12}, mask = data_mask).set_title('Correlations Among Features (in Competition Dataset)')

In [18]:
correlation_matrix = train_values_scaled.corr()

# Correlaciones mayores a 0.7
threshold = 0.7
high_correlations = correlation_matrix[(correlation_matrix > threshold) & (correlation_matrix < 1.0)]

high_correlations.dropna(axis=0, how='all', inplace=True)  # Elimina las filas que no tienen datos
high_correlations.dropna(axis=1, how='all', inplace=True)  # Elimina las columnas que no tienen datos

PlotMatrizCorrelacion(high_correlations)

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [19]:
def Dendrogram(X, threshold, title):
	X_T = X.T

	dist = sklearn.metrics.DistanceMetric.get_metric('euclidean')
	D = dist.pairwise(X_T)
	avD = np.average(D)
	print("Distancia Media\t {:6.2f}".format(avD))

	plt.figure(figsize=(12, 6))
	plt.title(title)
	plt.xlabel('Columnas')
	plt.ylabel('Distancia')

	clusters = cluster.hierarchy.linkage(D, method='complete')
	cluster.hierarchy.dendrogram(clusters, color_threshold=threshold, labels=X.columns.to_list(), leaf_rotation=90)
	plt.show()

In [20]:
Dendrogram(train_values_scaled, 180, 'Dendrograma de terremotos')

ValueError: Input contains NaN.

In [21]:
def ApplySelectKBest(X_train, X_val, y_train, k):
	selector = SelectKBest(f_regression, k=k)

	selector.fit_transform(X_train, y_train)
	selector.transform(X_val)

	print(f"Mejores {k} columnas = [")
	for col in selector.get_feature_names_out():
		print(f"  \'{col}\'")
	print("]\n")

In [22]:
ApplySelectKBest(X_train, X_val, y_train, 1)
ApplySelectKBest(X_train, X_val, y_train, 3)
ApplySelectKBest(X_train, X_val, y_train, 5)
ApplySelectKBest(X_train, X_val, y_train, 6)

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values